In [15]:
import pandas as pd
import numpy as np

In [27]:
def get_intents(excel_file):
    patient = pd.read_excel(excel_file, sheet_name=0)
    patient = patient.fillna("-")
    patient['text_all'] = patient.iloc[:,2:].apply(list, axis=1)
    patient_dict = patient.set_index('Informations')['text_all'].to_dict()
    patient_dict_no_nan = {}
    for key, value in patient_dict.items():
        value_no_nan = [x for x in value if x != "-"]
        patient_dict_no_nan[key] = value_no_nan

    sdd = pd.read_excel(excel_file, sheet_name=1)
    sdd = sdd.fillna('-')
    sdd['text_all'] = sdd.iloc[:,2:].apply(list, axis=1)
    sdd_dict = sdd[['Informations', 'type', 'text_all']].to_dict(orient='index')
    sdd_dict_no_nan = {}
    for key, value in sdd_dict.items():
        value_no_nan = [x for x in value['text_all'] if x != "-"]
        if value['Informations'] not in sdd_dict_no_nan.keys():
            sdd_dict_no_nan[value['Informations']] = {value['type']: value_no_nan}
        else:
            sdd_dict_no_nan[value['Informations']][value['type']] = value_no_nan
    sdd_list = []
    for key, value in sdd_dict_no_nan.items():
        tag = {}
        tag['tag'] = key
        tag.update(value)
        sdd_list.append(tag)
    return patient_dict_no_nan, sdd_list

patient_dict_no_nan, sdd_list = get_intents('import/ECOS Intents MV SDD328 I106.xlsx')
patient_dict_no_nan

{'name': ['M. Denis'],
 'taille': ['Je fais 1m75'],
 'poids': ['Je fais 70kg'],
 'job': ["Je suis artiste peintre. J'aime mon métier."],
 'motif': ["Je viens vous consulter parce que j'ai peur d'avoir la maladie de Parkinson..."],
 'stress': ["J'ai peur d'avoir la maladie de Parkinson.."],
 'age': ["J'ai 55 ans"],
 'antecedant': ["J'ai de l'hypertension. Je prends de l'AMLOR le matin."],
 'antecedantfam': ['Pas à ma connaissance', 'Je ne crois pas'],
 'howareyou': ['Ca va Docteur.', 'Un peu stressée mais ca va.'],
 'tabac': ['Non Docteur.']}

In [28]:
sdd_list

[{'tag': 'annonce',
  'patterns': ['Vous avez bien fait, nous allons prendre le temps de vous expliquer la maladie de Parkinson.',
   'En effet, vous avez des symptomes pouvant faire penser à une maladie de Parkinson',
   "A l'examen, je retrouve des elements pouvant en effet indiquer que vous êtes atteint de la maladie de Parkison."],
  'responses': ['Je vous ecoute Docteur.',
   "Dites moi en plus Docteur s'il vous plait."]},
 {'tag': 'explorationParkinson',
  'patterns': ["Vous présentez des tremblements, une lenteur aux mouvements et une rigidité qui peut faire penser à une maladie de Parkinson. il s'agit d'une maladie qui est due à un déficit d'une molécule nommé dopamine dans le cerveau.",
   "A l'examen, je retrouve des elements cliniques, comme des tremblements et une rigidité de vos mouvements. Il s'agit d'une maladie qui est due à un déficit d'une molécule nommé dopamine dans le cerveau."],
  'responses': ["Mais vous êtes sur ? Y'a pas besoin d'examen complémentaires ?"]},
 {